In [8]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
print(tf.__version__)

2.8.0


In [143]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

column_names = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare',
       'class', 'deck', 'embark_town', 'alone']

df = pd.read_csv(TRAIN_DATA_URL)
# pd.set_option('display.max_rows', None)
print(df)
# 데이터 정보 확인
print(df.info())
# 수치형 데이터 확인
print(df.describe())
# 범주형 데이터 확인
df.describe(include = np.object_)
# 결측치 확인
df.isnull().sum()

     survived     sex   age  n_siblings_spouses  parch     fare   class  \
0           0    male  22.0                   1      0   7.2500   Third   
1           1  female  38.0                   1      0  71.2833   First   
2           1  female  26.0                   0      0   7.9250   Third   
3           1  female  35.0                   1      0  53.1000   First   
4           0    male  28.0                   0      0   8.4583   Third   
..        ...     ...   ...                 ...    ...      ...     ...   
622         0    male  28.0                   0      0  10.5000  Second   
623         0    male  25.0                   0      0   7.0500   Third   
624         1  female  19.0                   0      0  30.0000   First   
625         0  female  28.0                   1      2  23.4500   Third   
626         0    male  32.0                   0      0   7.7500   Third   

        deck  embark_town alone  
0    unknown  Southampton     n  
1          C    Cherbourg     n

survived              0
sex                   0
age                   0
n_siblings_spouses    0
parch                 0
fare                  0
class                 0
deck                  0
embark_town           0
alone                 0
dtype: int64

In [144]:
# 데이터 요약

print("전체 데이터 수:", df.shape[0] * df.shape[1])
print(f"결측치 수: {df.isnull().sum().sum()}")
print("총 인원 수:", df["age"].count())
print("중복된 데이터:",df.duplicated().sum())

전체 데이터 수: 6270
결측치 수: 0
총 인원 수: 627
중복된 데이터: 69


In [152]:
feature_names = column_names[1:]
label_name = column_names[0]

print(feature_names)
print(label_name)
train_dataset = tf.data.experimental.make_csv_dataset(
    train_file_path,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

test_dataset = tf.data.experimental.make_csv_dataset(
    train_file_path,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)


features, labels = next(iter(train_dataset))

print(f"##features \n {features}")


['sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
survived
##features 
 OrderedDict([('sex', <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'male', b'male', b'male', b'male', b'female', b'female', b'male',
       b'female', b'male', b'male', b'female', b'female', b'female',
       b'female', b'male', b'female', b'female', b'male', b'female',
       b'male', b'male', b'male', b'female', b'male', b'male', b'male',
       b'male', b'male', b'male', b'female', b'male', b'male'],
      dtype=object)>), ('age', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([28.  , 38.  , 26.  , 34.5 , 39.  , 17.  , 23.  , 27.  , 57.  ,
        0.83, 28.  , 50.  , 19.  , 14.  , 28.  , 28.  , 22.  , 40.5 ,
       44.  , 27.  , 51.  ,  1.  , 28.  , 18.  , 29.  , 20.  , 33.  ,
       26.  , 40.  , 60.  , 28.  , 30.5 ], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(32,), dtype=int32, numpy=
array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,

In [153]:
# shuffle
train_dataset = train_dataset.shuffle(df.shape[0])

In [156]:
age_max = df['age'].max()
age_min = df['age'].min()
n_siblings_spouses_max = df['n_siblings_spouses'].max()
n_siblings_spouses_min = df['n_siblings_spouses'].min()
parch_max = df['parch'].max()
parch_min = df['parch'].min()
fare_max = df['fare'].max()
fare_min = df['fare'].min()


# 데이터 정규화
def normalization(features, labels):
    features['age'] = (features['age'] - age_min) / (age_max - age_min)
    features['n_siblings_spouses'] = (features['n_siblings_spouses'] - n_siblings_spouses_min) / (n_siblings_spouses_max - n_siblings_spouses_min)
    features['parch'] = (features['parch'] - parch_min) / (parch_max - parch_min)
    features['fare'] = (features['fare'] - fare_min) / (fare_max - fare_min)
    return features, labels

train_dataset = train_dataset.map(normalization)
test_dataset = test_dataset.map(normalization)
features, labels = next(iter(train_dataset))

print(f"features \n {features}")

features 
 OrderedDict([('sex', <tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'male', b'male', b'male', b'female', b'female', b'male', b'male',
       b'female', b'male', b'female', b'male', b'male', b'female',
       b'female', b'male', b'male', b'male', b'female', b'female',
       b'male', b'male', b'female', b'male', b'male', b'male', b'male',
       b'male', b'male', b'female', b'male', b'male', b'male'],
      dtype=object)>), ('age', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([0.49526814, 0.2933754 , 0.6214511 , 0.72239745, 0.3438486 ,
       0.3438486 , 0.3438486 , 0.3438486 , 0.43217665, 0.43217665,
       0.533123  , 0.30599368, 0.6466877 , 0.6214511 , 0.5709779 ,
       0.00315457, 0.3438486 , 0.5078864 , 0.20504732, 0.31861198,
       0.2681388 , 0.38170347, 0.35015774, 0.3438486 , 0.3438486 ,
       0.5205047 , 0.24921136, 0.3438486 , 0.3438486 , 0.3438486 ,
       0.38170347, 0.23028392], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(32,), 

In [157]:
# 문자열 처리 -> sex, deck, alone, class를 문자열로
CAT_COLUMNS = ['sex', 'deck', 'alone', 'class', 'embark_town']
NUM_COLUMNS = ['age', 'fare', 'n_siblings_spouses', 'parch']

feature_cols = []

# Create IndicatorColumn for categorical features
for feature in CAT_COLUMNS:
  vocab = df[feature].unique()
  feature_cols.append(tf.feature_column.indicator_column(
      tf.feature_column.categorical_column_with_vocabulary_list(feature, vocab)))

# Create NumericColumn for numerical features
for feature in NUM_COLUMNS:
  feature_cols.append(tf.feature_column.numeric_column(feature, dtype=tf.float32))

print(feature_cols)

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float

In [158]:
# 모델 생성
model = tf.keras.Sequential()
model.add(tf.keras.layers.DenseFeatures(feature_cols))
model.add(tf.keras.layers.Dense(8, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.fit(train_dataset, epochs=100)
        

Epoch 1/100


2022-05-19 03:55:10.388493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 1s 18ms/step - loss: 0.7251 - binary_accuracy: 0.3716
Epoch 2/100
20/20 [==============================] - 0s 13ms/step - loss: 0.7120 - binary_accuracy: 0.3700
Epoch 3/100
20/20 [==============================] - 0s 11ms/step - loss: 0.7019 - binary_accuracy: 0.3716
Epoch 4/100
20/20 [==============================] - 0s 11ms/step - loss: 0.6938 - binary_accuracy: 0.4179
Epoch 5/100
20/20 [==============================] - 0s 11ms/step - loss: 0.6871 - binary_accuracy: 0.5742
Epoch 6/100
20/20 [==============================] - 0s 10ms/step - loss: 0.6811 - binary_accuracy: 0.6938
Epoch 7/100
20/20 [==============================] - 0s 10ms/step - loss: 0.6754 - binary_accuracy: 0.7033
Epoch 8/100
20/20 [==============================] - 0s 10ms/step - loss: 0.6696 - binary_accuracy: 0.6635
Epoch 9/100
20/20 [==============================] - 0s 11ms/step - loss: 0.6639 - binary_accuracy: 0.6443
Epoch 10/100
20/20 [=============================

In [160]:
# 모델 평가하기
model.predict(test_dataset)

2022-05-19 03:55:40.202811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[0.22459026],
       [0.10759155],
       [0.8802296 ],
       [0.10866767],
       [0.12162338],
       [0.11828666],
       [0.1244107 ],
       [0.4401081 ],
       [0.10685172],
       [0.10630254],
       [0.49043557],
       [0.18720378],
       [0.46757835],
       [0.7872259 ],
       [0.69836456],
       [0.12753133],
       [0.21515702],
       [0.10934617],
       [0.09994785],
       [0.10654998],
       [0.10706964],
       [0.1250674 ],
       [0.10791885],
       [0.12359685],
       [0.9375549 ],
       [0.12692913],
       [0.6634556 ],
       [0.72677803],
       [0.813334  ],
       [0.29800594],
       [0.90366983],
       [0.19365498],
       [0.10627963],
       [0.11155994],
       [0.0953436 ],
       [0.6663874 ],
       [0.10613394],
       [0.5345392 ],
       [0.70957655],
       [0.8120674 ],
       [0.84365505],
       [0.4201832 ],
       [0.14704284],
       [0.1098848 ],
       [0.8615633 ],
       [0.56030136],
       [0.21979666],
       [0.107